In [45]:
import sys
import os 
sys.path.append(os.path.abspath(".."))

import numpy as np
import pandas as pd
import importlib
import matplotlib.pyplot as plt
from pyarrow import table

import volpy_func_lib as vp
import load_clean_lib
import table_lib
import vol_strat_lib as vs
import option_returns as orpy
from volpy_func_lib import return_df, Liquid_tickers, ALL_tickers
import volpy_func_ticker_lib as vtp
from volpy_func_ticker_lib import concat_ticker_datasets
# from volpy_func_lib import OEX_tickers, ALL_tickers, VIX_tickers, Cross_AM_tickers, Liquid_tickers

importlib.reload(vp)
importlib.reload(load_clean_lib)
importlib.reload(table_lib)
importlib.reload(vs)
importlib.reload(vtp)

os.makedirs("figures/Analysis", exist_ok=True)

## Liquid

In [46]:
name = "Liquid" #SPX_full, i2s1_full_v2', vix_markets_full
sum_df = vtp.concat_ticker_datasets(vp.Liquid_tickers, df_name="sum2")

_ = table_lib.CarrWu2009_table_2(sum_df, name = name)
_ = table_lib.CarrWu2009_table_3(sum_df, name = name)

Skipped 4/8 tickers: ['SPY', 'OEX', 'NDX', 'DJX']


## Cross-AM

In [47]:
name = "Cross-AM" #SPX_full, i2s1_full_v2', vix_markets_full
sum_df = vtp.concat_ticker_datasets(vp.Cross_AM_tickers, df_name="sum2")

_ = table_lib.CarrWu2009_table_2(sum_df, name = name)
_ = table_lib.CarrWu2009_table_3(sum_df, name = name)

## VIX

In [48]:
name = "VIX" #SPX_full, i2s1_full_v2', vix_markets_full
sum_df = vtp.concat_ticker_datasets(vp.VIX_tickers, df_name="sum2")

_ = table_lib.CarrWu2009_table_2(sum_df, name = name)
_ = table_lib.CarrWu2009_table_3(sum_df, name = name)

## OEX

In [49]:
name = "OEX" #SPX_full, i2s1_full_v2', vix_markets_full
sum_df = vtp.concat_ticker_datasets(vp.OEX_tickers, df_name="sum2")

_ = table_lib.CarrWu2009_table_2(sum_df, name = name)
_ = table_lib.CarrWu2009_table_3(sum_df, name = name)

Skipped 32/181 tickers: ['OEX', 'AGN', 'AVP', 'BDK', 'BHI', 'CELG', 'CGP', 'CMCSK', 'COV', 'CSC', 'EMC', 'ENE', 'EP', 'GBLX', 'HET', 'HNZ', 'HSH', 'LEH', 'MAY', 'MON', 'NT', 'NXTL', 'NYX', 'OMX', 'ONE', 'PNU', 'RAL', 'RSH', 'RTN', 'SGP', 'TOY', 'UBAN']
